## Introduction

Internally, all [Findr.jl][1] functions work with matrices or array-based data, and the [DataFrame](https://dataframes.juliadata.org/stable/) based `findr` methods used in the [coexpression analysis](coexpression.qmd), [association analysis](association.qmd), and [causal inference](causal-inference.qmd) tutorials are wrapper functions provided for convenience. If you prefer matrix-based data over DataFrames, you can directly use matrix-based `findr` methods without having to create DataFrames first.

## Set up the environment


In [ ]:
using DrWatson
quickactivate(@__DIR__)

using DataFrames
using Arrow

using Findr

## Load data

Let's pretend our GEUVADIS data is in a matrix-based format:


In [ ]:
Xt = Matrix(DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dt.arrow"))));
Xm = Matrix(DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dm.arrow"))));
Gm = Matrix(DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dgm.arrow"))));

We also need the microRNA eQTL mapping (see the [causal inference tutorial](causal-inference.qmd)), in this case in the form of an array where each row corresponds to a cis-eQTL/eGene pair represented by of a column index of `Gm` (i.e. a SNP) and a column index of `Xm` (i.e. a microRNA). [Recall](causal-inference.qmd) that due to the preprocessing of the [findr-geuvadis][2] data. the column indices are identical, but this will not be the case in general:


In [ ]:
mirpairs = zeros(Int32,size(Gm,2),2);
for k=1:size(mirpairs,1)
    mirpairs[k,:] = [k k]
end

Note that data must be stored in matrices where **columns correspond to variables** (genes, SNPs, etc.) and **rows correspond to observations** (samples).

## Run Findr.jl

Below, we only show the relevant `findr` commands. Check the corresponding tutorials and [Findr.jl documentation][6] for more details. 

### Coexpression analysis

#### All-vs-all

Coexpression analysis on a single matrix returns a square matrix with dimensions equal to the number of variables (columns) in the input matrix:


In [ ]:
P = findr(Xm)

In the output, columns correspond to A-genes (causal factors) and rows to B-genes (targets), that is:

$$
P_{i,j} = P(X_j \to X_i)
$$

Note that the diagonal is arbitrarily set to one, Findr cannot make any inferences about the presence or absence of self-regulation!

#### Bipartite

Analyse coexpression *from* a subset of variables *to* the whole set:


In [ ]:
P = findr(Xm; cols=[1,3,7,50])

Analyse coexpression *from* the variables in `Xm` *to* the variables in `Xt`:


In [ ]:
P = findr(Xt,Xm)

### Association analysis

Testing associations between eQTL genotypes in `Gm`and microRNA expression levels in `Xm`:


In [ ]:
P = findr(Xm,Gm)

In the output, columns correspond to eQTLs and rows to genes, that is,


$$
P_{i,j} = P(E_j \to X_i)
$$

### Causal inference

#### Subset-to-all

When you run causal inference with `findr` using matrix-based inputs, the default is to return posterior probabilities for [each test](https://tmichoel.github.io/Findr.jl/dev/realLLR/) separately:


In [ ]:
P = findr(Xm,Gm,mirpairs);

Note the dimensions of `P`:


In [ ]:
size(P)

The third dimension indexes the A-genes (causes), the second dimension the tests (test 2-5, see link above), and the first the B-genes (targets). If you are interested only in a specific combination, use the optional `combination` argument as explained in the [causal inference tutorial](causal-inference.qmd):


In [ ]:
P = findr(Xm,Gm,mirpairs; combination="IV");

[1]: https://github.com/tmichoel/Findr.jl
[2]: https://github.com/lingfeiwang/findr-data-geuvadis
[3]: https://doi.org/10.1038/nature12531
[4]: https://dataframes.juliadata.org/stable/
[5]: https://doi.org/10.1371/journal.pcbi.1005703
[6]: https://tmichoel.github.io/Findr.jl/dev/